In [3]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
data=pd.read_csv("Companies.csv")
companynames=list(data.iloc[:,0])
links=list(data.iloc[:,1])
import numpy as np
from tqdm.notebook import tqdm
#links for old format page
linkso=[]
for i in range(0,len(links)):
    a=links[i].find('VI')
    b=links[i][:a]+links[i][a+2:]
    linkso.append(b)

#links for profit-loss page
linkso_pl=[]
for i in range(0,len(linkso)):
    a=linkso[i].find('balance-sheet')
    b=linkso[i][:a]+'profit-loss'+linkso[i][a+13:]
    linkso_pl.append(b)

In [9]:
linkso_pl

['https://www.moneycontrol.com/financials/adaniportsspecialeconomiczone/profit-loss/MPS#MPS',
 'https://www.moneycontrol.com/financials/asianpaints/profit-loss/AP31#AP31',
 'https://www.moneycontrol.com/financials/bajajauto/profit-loss/BA10#BA10',
 'https://www.moneycontrol.com/financials/bajajfinance/profit-loss/BAF#BAF',
 'https://www.moneycontrol.com/financials/bhartiairtel/profit-loss/BA08#BA08',
 'https://www.moneycontrol.com/financials/bharatpetroleumcorporation/profit-loss/BPC#BPC',
 'https://www.moneycontrol.com/financials/britanniaindustries/profit-loss/BI#BI',
 'https://www.moneycontrol.com/financials/cipla/profit-loss/C#C',
 'https://www.moneycontrol.com/financials/coalindia/profit-loss/CI11#CI11',
 'https://www.moneycontrol.com/financials/drreddyslaboratories/profit-loss/DRL#DRL',
 'https://www.moneycontrol.com/financials/eichermotors/profit-loss/EM#EM',
 'https://www.moneycontrol.com/financials/gailindia/profit-loss/GAI#GAI',
 'https://www.moneycontrol.com/financials/grasi

In [4]:
''' Scrapping Net Sales'''
#As the details of net sales was not provided, the following banks were excluded from the file:
#HDFC bank, ICICI bank, Indusland bank, Kotak Mahindra Bank, Yes Bank, SBI
#As interestt cover ratio was not provided, Bajaj Finserv and Infosys were also excluded.

netsales_list=[]
for i in tqdm(range(0,len(linkso_pl))):
    pageold=requests.get(linkso_pl[i])
    soup = BeautifulSoup(pageold.text, 'html.parser')
    about=soup.findAll('table')
    about=str(about)
    a=re.findall('<td>Net Sales</td>\n.*</td>',about)
    netsales=re.findall('>[0-9].*<',str(a))
    netsales=netsales[0][1:len(netsales[0])-1]
    if(len(netsales)>6):
        a=netsales.find(',')
        netsales_list.append(float(netsales[:a]+netsales[a+1:]))
    else:
        netsales_list.append(float(netsales))

In [10]:
netsales_list

[4643.28,
 16391.78,
 29918.65,
 17383.97,
 54317.1,
 284382.95,
 10986.68,
 12659.15,
 934.3,
 11850.4,
 9794.48,
 75126.3,
 20550.43,
 32606.0,
 43348.04,
 33650.54,
 45749.16,
 38785.0,
 15407.35,
 6738.3,
 527692.69,
 44995.65,
 64262.0,
 86987.86,
 53614.0,
 75610.6,
 90307.43,
 109609.42,
 34119.12,
 335978.0,
 12531.93,
 69202.76,
 70610.71,
 131306.0,
 29225.4,
 20009.64,
 40649.17,
 9641.0,
 35858.0,
 50407.0,
 6857.9]

In [11]:
'''  The Net Sales of each company is saved in the netsales list in Crores.
     Now filtering out the companies with less than 250 cr. Net Sales.'''
filtersales=[]
filtercomp=[]
filterlink=[]
filterolink=[]
for i in tqdm(range(0,len(netsales_list))):
    if(netsales_list[i]<250.00):
        filtersales.append(netsales_list[i])
        filtercomp.append(companynames[i])
        filterlink.append(links[i])
        filterolink.append(linkso[i])
for i in tqdm(range(0,len(filtersales))):
    netsales_list.remove(filtersales[i])
    companynames.remove(filtercomp[i])
    links.remove(filterlink[i])
    linkso.remove(filterolink[i])

In [15]:
filterolink

[]

In [16]:
''' Scraping debt equity Ratio '''

#links for ratio page
linkso_r=[]
for i in tqdm(range(0,len(linkso))):
    a=linkso[i].find('balance-sheet')
    b=linkso[i][:a]+'ratios'+linkso[i][a+13:]
    linkso_r.append(b)
debtequity_ratio=[]
for i in tqdm(range(0,len(linkso_r))):
    pageold=requests.get(linkso_r[i])
    soup = BeautifulSoup(pageold.text, 'html.parser')
    about=soup.findAll('table')
    about=str(about)
    a=re.findall('<td>Debt Equity Ratio</td>\n.*</td>',about)
    der=re.findall('>[0-9].*<',str(a))
    if(len(der)==0):
        der=0
    else:
        der=der[0][1:len(der[0])-1]
    debtequity_ratio.append(float(der))


In [10]:
#Companies with Debt Equity Ratio greater than 0.3 are filtered 

filtersales=[]  
filterder=[]
filtercomp=[]
filterlink=[]
filterolink=[]
for i in range(0,len(debtequity_ratio)):
    if(debtequity_ratio[i]>0.3):
        filterder.append(debtequity_ratio[i])
        filtercomp.append(companynames[i])
        filterlink.append(links[i])
        filterolink.append(linkso[i])
        filtersales.append(netsales_list[i])
for i in range(0,len(filterder)):
    debtequity_ratio.remove(filterder[i])
    companynames.remove(filtercomp[i])
    links.remove(filterlink[i])
    linkso.remove(filterolink[i])
    netsales_list.remove(filtersales[i])


In [11]:
''' Scraping Interesr Coverage Ratio '''
#links for ratio page
linkso_r=[]
for i in range(0,len(linkso)):
    a=linkso[i].find('balance-sheet')
    b=linkso[i][:a]+'ratios'+linkso[i][a+13:]
    linkso_r.append(b)

icr_list=[]
for i in range(0,len(linkso_r)): 
    pageold=requests.get(linkso_r[i])
    soup = BeautifulSoup(pageold.text, 'html.parser')
    about=soup.findAll('table')
    about=str(about)
    a=re.findall('<td>Interest Cover</td>\n.*</td>',about)
    icr=re.findall('>[0-9].*<',str(a))
    icr=icr[0][1:len(icr[0])-1]
    if(len(icr)>6):
        a=icr.find(',')
        icr_list.append(float(icr[:a]+icr[a+1:]))
    else:
        icr_list.append(float(icr))


filtericr=[]
filtercomp=[]
filterlink=[]
filterolink=[]
filtersales=[]
filterder=[]
for i in range(0,len(icr_list)):
    if(icr_list[i]<4):
        filtericr.append(icr_list[i])
        filtercomp.append(companynames[i])
        filterlink.append(links[i])
        filterolink.append(linkso[i])
        filtersales.append(netsales_list[i])
        filterder.append(debtequity_ratio[i])
for i in range(0,len(filtericr)):
    icr_list.remove(filtericr[i])
    companynames.remove(filtercomp[i])
    links.remove(filterlink[i])
    linkso.remove(filterolink[i])   
    netsales_list.remove(filtersales[i])
    debtequity_ratio.remove(filterder[i])

In [12]:
''' Scraping Total Income'''

links_pl=[]
for i in range(0,len(links)):
    a=links[i].find('balance-sheet')
    b=links[i][:a]+'profit-loss'+links[i][a+13:]
    links_pl.append(b)

ti_list=[]
for i in range(0,len(links_pl)):
    pageold=requests.get(links_pl[i])
    soup = BeautifulSoup(pageold.text, 'html.parser')
    about=soup.findAll('table')
    about=str(about)
    a=re.findall('<td>Total Revenue</td>\n.*</td>',about)
    ti=re.findall('>[0-9].*<',str(a))
    ti=ti[0][1:len(ti[0])-1]
    if(len(ti)>6):
        a=ti.find(',')
        ti_list.append(float(ti[:a]+ti[a+1:]))
    else:
        ti_list.append(float(ti))


In [13]:
''' Scraping Equity Share Capital'''
tot_assets=[]
current_lia=[]
noncurrent_lia=[]

for i in range(0,len(links)):
    pageold=requests.get(links[i])
    soup = BeautifulSoup(pageold.text, 'html.parser')
    about=soup.findAll('table')
    about=str(about)
    a=re.findall('<td>Total Assets</td>\n.*</td>',about)
    totas=re.findall('>[0-9].*<',str(a))
    totas=totas[0][1:len(totas[0])-1]
    if(len(totas)>6):
        a=totas.find(',')
        tot_assets.append(float(totas[:a]+totas[a+1:]))
    else:
        tot_assets.append(float(totas))
    
    a=re.findall('<td>Total Non-Current Liabilities</td>\n.*</td>',about)
    totncl=re.findall('>[0-9].*<',str(a))
    totncl=totncl[0][1:len(totncl[0])-1]
    if(len(totncl)>6):
        a=totncl.find(',')
        noncurrent_lia.append(float(totncl[:a]+totncl[a+1:]))
    else:
        noncurrent_lia.append(float(totncl))
    
    a=re.findall('<td>Total Current Liabilities</td>\n.*</td>',about)
    totcl=re.findall('>[0-9].*<',str(a))
    totcl=totcl[0][1:len(totcl[0])-1]
    if(len(totcl)>6):
        a=totcl.find(',')
        current_lia.append(float(totcl[:a]+totcl[a+1:]))
    else:
        current_lia.append(float(totcl))
    
roe_list=np.array(ti_list)/(np.array(tot_assets)-np.array(current_lia)-np.array(noncurrent_lia))
roe_list=roe_list.tolist()

In [14]:
''' Scraping for PE'''

links_r=[]
for i in range(0,len(links)):
    a=links[i].find('balance-sheet')
    b=links[i][:a]+'ratios'+links[i][a+13:]
    links_r.append(b)

pricbv=[]
bkvshr=[]
for i in range(0,len(links_r)):
    pageold=requests.get(links_r[i])
    soup = BeautifulSoup(pageold.text, 'html.parser')
    about=soup.findAll('table')
    about=str(about)
    a=re.findall('<td>Price/BV.*</td>\n.*</td>',about)
    pbv=re.findall('>[0-9].*<',str(a))
    pbv=pbv[0][1:len(pbv[0])-1]
    if(len(pbv)>6):
        a=pbv.find(',')
        pricbv.append(float(pbv[:a]+pbv[a+1:]))
    else:
        pricbv.append(float(pbv))
        
    a=re.findall('<td>Book Value.*/Share.*</td>\n.*</td>',about)
    a=a[0]
    pbvs=re.findall('>[0-9].*<',str(a))
    pbvs=pbvs[0][1:len(pbvs[0])-1]
    if(len(pbvs)>6):
        a=pbvs.find(',')
        bkvshr.append(float(pbvs[:a]+pbvs[a+1:]))
    else:
        bkvshr.append(float(pbvs))

eps_list=[]
for i in range(0,len(links_pl)):
    pageold=requests.get(links_pl[i])
    soup = BeautifulSoup(pageold.text, 'html.parser')
    about=soup.findAll('table')
    about=str(about)
    a=re.findall('<td>Basic EPS.*</td>\n.*</td>',about)
    eps=re.findall('>[0-9].*<',str(a))
    eps=eps[0][1:len(eps[0])-1]
    if(len(eps)>6):
        a=eps.find(',')
        eps_list.append(float(eps[:a]+eps[a+1:]))
    else:
        eps_list.append(float(eps))


pe=np.array(bkvshr)*np.array(pricbv)/np.array(eps_list)
pe=pe.tolist()

In [15]:
''' Filter PE values '''
filtersales=[]  
filterder=[]
filtericr=[]
filterroe=[]
filterpe=[]
filtercomp=[]
filterlink=[]
filterolink=[]
for i in range(0,len(pe)):
    if(pe[i]>25):
        filtersales.append(netsales_list[i])
        filterder.append(debtequity_ratio[i])
        filtericr.append(icr_list[i])
        filterroe.append(roe_list[i])
        filterpe.append(pe[i])
        filtercomp.append(companynames[i])
        filterlink.append(links[i])

for i in range(0,len(filterpe)):
    netsales_list.remove(filtersales[i])
    debtequity_ratio.remove(filterder[i])
    icr_list.remove(filtericr[i])
    roe_list.remove(filterroe[i])
    pe.remove(filterpe[i])
    companynames.remove(filtercomp[i])
    links.remove(filterlink[i])

In [16]:
final=list(zip(companynames,netsales_list,debtequity_ratio,icr_list,roe_list,pe))

filtered_list=pd.DataFrame(final,columns=['Company','Net Sales in cr.','Debt to Equity Ratio','Interest Coverage Ratio','Return On Equity (ROE)','P/E Ratio'])


In [17]:
filtered_list

       Company  Net Sales in cr.  Debt to Equity Ratio  \
0   BAJAJ-AUTO          30249.96                  0.02   
1        CIPLA          12374.01                  0.00   
2    COALINDIA            934.30                  0.00   
3         GAIL          75126.30                  0.00   
4      HCLTECH          26012.00                  0.00   
5   HEROMOTOCO          33650.54                  0.07   
6     INFRATEL           6821.70                  0.00   
7          M&M          53614.00                  0.11   
8         ONGC         109609.42                  0.00   
9        TECHM          27219.60                  0.00   
10       WIPRO          48123.80                  0.10   

    Interest Coverage Ratio  Return On Equity (ROE)  P/E Ratio  
0                   1420.90                1.464620  18.024956  
1                    147.90                0.820657  22.553348  
2                    586.50                0.820364  14.040622  
3                     68.93                